# Transferência de Conhecimento - Transfer Learning

**Objetivo**: construir um modelo de classificação de cachorros e gatos com base em outros modelos pré-treinados.

In [ ]:
import os
import warnings
import numpy as np
import tensorflow as tf
from keras.models import Model
from google.colab import drive
import matplotlib.pyplot as plt
from keras.applications import VGG16
from keras.preprocessing import image
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

# ignorar os warnings
warnings.filterwarnings('ignore')

In [ ]:
# conectar colab no google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# definindo algumas constantes
DATA_PATH = '/content/drive/MyDrive/Bootcamp_DataScience/DeepLearningTensorflow/data/dataset'
TRAIN_PATH = os.path.join(DATA_PATH, 'training_set')
TEST_PATH = os.path.join(DATA_PATH, 'test_set')
BATCH_SIZE = 64
IMG_STD_SIZE = 224

In [ ]:
# criando um data augmentation para base de treinamento
train_datagen = ImageDataGenerator(
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

# para a base de teste, vamos apenas padronizar os pixels
test_datagen = ImageDataGenerator(
    rescale=1/255
)

In [ ]:
# vamos construir DataLoaders (carregar imagens pouco a pouco na memória)
train_set = train_datagen.flow_from_directory(
    directory=TRAIN_PATH,
    target_size=(IMG_STD_SIZE, IMG_STD_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
test_set = test_datagen.flow_from_directory(
    directory=TEST_PATH,
    target_size=(IMG_STD_SIZE, IMG_STD_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
# instanciando uma rede pré-treinada (aqui estamos usando o conhecimento prévio)
vgg = VGG16(
    input_shape=(IMG_STD_SIZE, IMG_STD_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

58889256/58889256 [==============================] - 4s 0us/step


In [ ]:
# visualizando arquitetura
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# adicionando camadas totalmente conectadas (MLP) - isso vai diferenciar a nossa
# rede da VGG16 original
def create_dense_model(vgg, neuron_list):
  # obtendo saídas da vgg e passando pelo flatten
  x = vgg.output
  x = Flatten()(x)

  # adicionando camadas MLP
  for n in neuron_list:
    x = Dense(n, activation='relu')(x)

  # adicionando última camada
  out = Dense(1, activation='sigmoid')(x)

  # juntando tudo num modelp
  model = Model(inputs=vgg.inputs, outputs=out)

  # impedindo o treinamento das camadas convolucionais
  for layer in model.layers[:20]:
    layer.trainable = False

  # compilar o modelo
  model.compile(
      optimizer='adam',
      loss='binary_crossentropy',
      metrics=['accuracy']
  )

  return model

In [ ]:
# analisando um primeiro modelo
cnn_model = create_dense_model(vgg, neuron_list=[4000, 256])
cnn_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# treinamento da rede
history = cnn_model.fit(train_set, batch_size=BATCH_SIZE, epochs=30,
                        initial_epoch=0, validation_data=test_set)


Epoch 1/30
125/125 [==============================] - 4779s 38s/step - loss: 1.2249 - accuracy: 0.7579 - val_loss: 0.2658 - val_accuracy: 0.8810
Epoch 2/30
125/125 [==============================] - 148s 1s/step - loss: 0.3023 - accuracy: 0.8646 - val_loss: 0.2218 - val_accuracy: 0.9060
Epoch 3/30
125/125 [==============================] - 152s 1s/step - loss: 0.2718 - accuracy: 0.8810 - val_loss: 0.2074 - val_accuracy: 0.9170
Epoch 4/30
125/125 [==============================] - 150s 1s/step - loss: 0.2734 - accuracy: 0.8801 - val_loss: 0.2091 - val_accuracy: 0.9125
Epoch 5/30
125/125 [==============================] - 149s 1s/step - loss: 0.2511 - accuracy: 0.8917 - val_loss: 0.2395 - val_accuracy: 0.8960
Epoch 6/30
125/125 [==============================] - 152s 1s/step - loss: 0.2573 - accuracy: 0.8905 - val_loss: 0.2016 - val_accuracy: 0.9135
Epoch 7/30
125/125 [==============================] - 148s 1s/step - loss: 0.2340 - accuracy: 0.9011 - val_loss: 0.2613 - val_accuracy: 0.88

In [ ]:
model_name = 'vgg16_4000_256'

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(1, 2, 1)
plt.plot(range(1, 30+1), history.history['accuracy'], label='Treinamento')
plt.plot(range(1, 30+1), history.history['val_accuracy'], label='Teste')
plt.legend(loc='best')
plt.xlabel('Número de Épocas')
plt.ylabel('Acurácia')
plt.title(f'Curva de Aprendizado - Acurácia')

ax = fig.add_subplot(1, 2, 2)
plt.plot(range(1, 50+1), history.history['loss'], label='Treinamento')
plt.plot(range(1, 50+1), history.history['val_loss'], label='Teste')
plt.legend(loc='best')
plt.xlabel('Número de Épocas')
plt.ylabel('Função de Perda')
plt.title('Curva de Aprendizado - Função de Perda')
plt.save_fig(os.path.join(DATA_PATH, 'models', f'{model_name}.png'), dpi=300, bboc_inches='tight')

NameError: ignored

In [ ]:
# serialize model to JSON
model_json = cnn_model.to_json()
with open(os.path.join(DATA_PATH, 'models', f"{model_name}.json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn_model.save_weights(os.path.join(DATA_PATH, 'models', f"{model_name}.h5"))